In [1]:
import os
from typing import Annotated, Sequence, TypedDict
from langchain_core.tools import Tool
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import WebBaseLoader, ArxivLoader, TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

from langgraph.prebuilt import create_react_agent
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import BaseMessage, HumanMessage
from langgraph.graph.message import add_messages
from langchain_anthropic import ChatAnthropic

/Users/akashdeepsangwan/Desktop/Code/GenAI/AgenticAI/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
api_key= os.getenv("claudeAPI")

llm = ChatAnthropic(
    model = "claude-haiku-4-5-20251001",
    temperature = 0.7,
    api_key = api_key
)

Embeddings = HuggingFaceEmbeddings(
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
)


In [3]:
urls=[
    "https://docs.langchain.com/oss/python/langgraph/overview",
    "https://docs.langchain.com/oss/python/langgraph/workflows-agents",
    "https://docs.langchain.com/oss/python/langgraph/graph-api#map-reduce-and-the-send-api"
]

docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

chunks = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 100).split_documents(docs_list) # each element is a doc
vector_store = FAISS.from_documents(chunks,Embeddings)
retriever = vector_store.as_retriever()


In [4]:
result =retriever.invoke("what is langgraph ?")
print(result[0].page_content)

​Acknowledgements
LangGraph is inspired by Pregel and Apache Beam. The public interface draws inspiration from NetworkX. LangGraph is built by LangChain Inc, the creators of LangChain, but can be used without LangChain.


In [5]:
from langchain_core.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "retriever_vector_db_blog",
    "Search and rin information about LangGraph"
)


### Tool is working

In [6]:
retriever_tool.invoke("What is langGraph?")

'\u200bAcknowledgements\nLangGraph is inspired by Pregel and Apache Beam. The public interface draws inspiration from NetworkX. LangGraph is built by LangChain Inc, the creators of LangChain, but can be used without LangChain.\n\nAt its core, LangGraph models agent workflows as graphs. You define the behavior of your agents using three key components:\n\nprint(f"Step: {metadata[\'langgraph_step\']}")\n    print(f"Node: {metadata[\'langgraph_node\']}")\n    print(f"Triggers: {metadata[\'langgraph_triggers\']}")\n    print(f"Path: {metadata[\'langgraph_path\']}")\n    print(f"Checkpoint NS: {metadata[\'langgraph_checkpoint_ns\']}")\n\n    return state\n\n\u200bVisualization\nIt’s often nice to be able to visualize graphs, especially as they get more complex. LangGraph comes with several built-in ways to visualize graphs. See this how-to guide for more info.\n\nEdit this page on GitHub or file an issue.\nConnect these docs to Claude, VSCode, and more via MCP for real-time answers.Was this

In [7]:
langchain_url  = [
    "https://docs.langchain.com/oss/python/langchain/overview",
    "https://docs.langchain.com/oss/python/langchain/rag",
]

web_doc_load = [WebBaseLoader(url).load() for url in langchain_url]
langchain_docs = [doc for sublist in web_doc_load for doc in sublist]
chunks_lang = RecursiveCharacterTextSplitter(chunk_size =1000, chunk_overlap = 100).split_documents(langchain_docs)
vb = FAISS.from_documents(chunks_lang,Embeddings)
langchain_retriever = vb.as_retriever()


In [8]:
langchain_tool  = create_retriever_tool(
    langchain_retriever,
    "retriever_vector_langchain_blog",
    "Search and run information about LangChain"
)


In [9]:
tools = [retriever_tool, langchain_tool]

### LangGraph workflow

In [10]:
class AgentState(TypedDict) :
    messages : Annotated[Sequence[BaseMessage], add_messages]


## Nodes

In [11]:
def agent(state) :
    """ 
    Invokes the agent model to generate a response based in the current sate. Given the question, 
    it will decide the retrieve using the retriever_tool, or simply end.

    Args: 
    state(messages) : The current state

    Returns :
        dict : The updated state with the agent response attended to messages
    """

    print("--- call agent -----")
    messages = state["messages"]

    api_key= os.getenv("claudeAPI")

    llm = ChatAnthropic(
        model = "claude-haiku-4-5-20251001",
        temperature = 0.7,
        api_key = api_key
    )


    llm= llm.bind_tools(tools)
    response = llm.invoke(messages)
    
    # We return a list, because this will get added to the existing list
    return {"messages" : [response]}

In [14]:
from typing import Literal, Annotated, Sequence
from typing_extensions import TypedDict

from langchain_core.messages import BaseMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field


### Grade_Documents Node

In [21]:
from langchain_core.prompts.string import PromptTemplateFormat


def grade_documents(state) -> Literal["generate", 'rewrite'] :
    """  
    Determines Whether the retrieved documnets are relevant to the question
    
    Args :
        state(messages) : The Current state

    Returns :
        str : A decision for whether the documents are relevant or not 
    """
    print(" --- check relevance")

    # Data Model
    class grade(BaseModel) :
        """ Binary Score for relevance check """
        binary_score : str = Field(description = "Relevance score 'yes' or 'no' ")


    api_key= os.getenv("claudeAPI")

    llm = ChatAnthropic(
        model = "claude-haiku-4-5-20251001",
        temperature = 0.7,
        api_key = api_key
    )
    

    llm_with_tool = llm.with_structured_output(grade)   

    prompt = PromptTemplate(
        template = """ 
        You are a grader assessing relevance of a retrieved document to a user question.
        Here is the retrieved document : \n\n {context} \n\n
        Here is the user question : {question} \n

        If the document contains keywords(s) or semantic meaning related to the user question, grade it as relevant. \n
        Give Binary Score 'yes' or 'no' score to indicate whether the document is relevant to the question. 
        """,

        input_variables= ["context", "question"],
    )

    chain = prompt | llm_with_tool

    messages = state['messages']
    last_message = messages[-1]

    # The first question that is being asked will be appended here
    question = messages[0].content
    docs = last_message.content


    scored_result = chain.invoke({"question" : question, "context" : docs})
    score = scored_result.binary_score

    if score == "yes" :
        print("-- Decision : DOCS Relevant")
        return "generate"    # it will go to the generate node


     
        


### Generate Node

In [17]:
from langsmith import Client
def generate(state) :
    """ 
    Generate answer

    Args :
        State(messages) : The current state

    Returns :
        dict : The updated message
    """
    print("--- Generate----")
    messages = state["messages"]
    question = messages[0].content
    last_message = messages[-1]

    docs = last_message.content

    # prompt (use LangSmith; langchainhub is deprecated)
    
    client = Client()
    prompt = client.pull_prompt("rlm/rag-prompt")

    #LLM - already initialised in the other tab
    api_key= os.getenv("claudeAPI")

    llm = ChatAnthropic(
        model = "claude-haiku-4-5-20251001",
        temperature = 0.7,
        api_key = api_key
    )
    
    # Post-processing
    
    def format_docs(docs) :
        return "\n\n".join(docs.page_content for doc in docs)
    
    #chain
    rag_chain = prompt | llm | StrOutputParser()


    # Run
    response = rag_chain.invoke({"context" : docs, "question" : question})
    return {"messages" : [response]}

### ReWrite Node

In [18]:
def rewrite(state) :
    
    """ 
    Transform the query to produce the better answer

    Args : 
        state (messages) : The current state
    
    Returns :
        dict : The updated state with re-phrased question    
    """

    print("----Transform Query----")
    messages = state['messages']
    question = messages[0].content


    api_key= os.getenv("claudeAPI")

    llm = ChatAnthropic(
        model = "claude-haiku-4-5-20251001",
        temperature = 0.7,
        api_key = api_key
    )

    msg = [
        HumanMessage(
            content = """
            Look at the input and try to reason about the underlying semantic intent/meaning.
            Here is the initial question :
            \n ------- \n
            {question}
            \n --------- \n
            Formulate an improved question : 
            """
        )
    ]

    response = llm.invoke(msg)
    return {"messages" : [response]}





### Graph WorkFlow

In [19]:
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition


workflow = StateGraph(AgentState)


#Nodes
workflow.add_node("agent", agent)
retrieve = ToolNode(tools)
workflow.add_node("retrieve", retrieve)
workflow.add_node("rewrite", rewrite) # Re-writing the question
workflow.add_node("generate", generate)


# Edges
workflow.add_edge(START, "agent")

# Decide whether to retrieve
workflow.add_conditional_edges(
    "agent", tools_condition, {"tools" : "retrieve", END : END},
)


# Edge taken after the 'action' node is called. 
workflow.add_conditional_edges(
    "retrieve", grade_documents
)
workflow.add_edge("generate", END)
workflow.add_edge("rewrite", "agent")

# compile
graph = workflow.compile()

In [22]:
graph.invoke({"messages": "what is langgraph ?"})


--- call agent -----
-- Decision : DOCS Relevant
--- Generate----


{'messages': [HumanMessage(content='what is langgraph ?', additional_kwargs={}, response_metadata={}, id='52dc339c-90f2-48bb-a008-705b9da41f2a'),
  AIMessage(content=[{'id': 'toolu_015bX4eP1fRNBUw3fvZXw52v', 'input': {'query': 'what is langgraph'}, 'name': 'retriever_vector_db_blog', 'type': 'tool_use'}], additional_kwargs={}, response_metadata={'id': 'msg_01SBPngeTvV6WsWdzYDki9m4', 'model': 'claude-haiku-4-5-20251001', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'cache_creation': {'ephemeral_1h_input_tokens': 0, 'ephemeral_5m_input_tokens': 0}, 'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 658, 'output_tokens': 63, 'server_tool_use': None, 'service_tier': 'standard'}, 'model_name': 'claude-haiku-4-5-20251001', 'model_provider': 'anthropic'}, id='lc_run--019c078f-86a9-7751-ac37-d03596f7733d-0', tool_calls=[{'name': 'retriever_vector_db_blog', 'args': {'query': 'what is langgraph'}, 'id': 'toolu_015bX4eP1fRNBUw3fvZXw52v', 'type': 'tool_c

In [23]:
graph.invoke({"messages" : "what is langchain?"})

--- call agent -----
-- Decision : DOCS Relevant
--- Generate----


{'messages': [HumanMessage(content='what is langchain?', additional_kwargs={}, response_metadata={}, id='7ed5002d-68d4-49ed-926b-7de1b07d8c22'),
  AIMessage(content=[{'id': 'toolu_01MNFjQshrSWQHvUW6uJZRj6', 'input': {'query': 'what is langchain'}, 'name': 'retriever_vector_langchain_blog', 'type': 'tool_use'}], additional_kwargs={}, response_metadata={'id': 'msg_011dTxNpPfo2W85ZZfT474tT', 'model': 'claude-haiku-4-5-20251001', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'cache_creation': {'ephemeral_1h_input_tokens': 0, 'ephemeral_5m_input_tokens': 0}, 'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 657, 'output_tokens': 63, 'server_tool_use': None, 'service_tier': 'standard'}, 'model_name': 'claude-haiku-4-5-20251001', 'model_provider': 'anthropic'}, id='lc_run--019c0790-6178-7652-9b91-ce92d153ea39-0', tool_calls=[{'name': 'retriever_vector_langchain_blog', 'args': {'query': 'what is langchain'}, 'id': 'toolu_01MNFjQshrSWQHvUW6uJZRj6', 't

#### Not calling any tool here, it is directly using the LLM

In [24]:
graph.invoke({"messages": "what is Machine Learning ?"})

--- call agent -----


{'messages': [HumanMessage(content='what is Machine Learning ?', additional_kwargs={}, response_metadata={}, id='11812ca8-41b4-4e57-832b-5320675284d9'),
  AIMessage(content="Machine Learning is a branch of artificial intelligence (AI) that focuses on enabling computers to learn and improve from experience without being explicitly programmed for every scenario.\n\nHere are the key aspects of Machine Learning:\n\n## Core Concept\nMachine Learning systems learn patterns from data and use those patterns to make predictions or decisions on new, unseen data. Instead of following pre-written rules, ML algorithms identify rules and patterns automatically.\n\n## Main Types of Machine Learning\n\n1. **Supervised Learning** - Learning from labeled data where the correct answers are provided\n   - Examples: Classification, Regression\n   - Used for: Predicting house prices, spam detection, image recognition\n\n2. **Unsupervised Learning** - Learning from unlabeled data to find hidden patterns\n   